In [1]:
using JuMP, Ipopt, ForwardDiff, LinearAlgebra
using Plots, LaTeXStrings
pyplot()

∇(f,x) = ForwardDiff.gradient(f,x)
∇²(f,x) = ForwardDiff.hessian(f,x)

∇² (generic function with 1 method)

In [2]:
function SQP()
        # Problem information
        n = 2 # total of variables
        m = 4 # total of ineq. constraints

        f(x) =  2x[1]^2 + 2x[2]^2 - 2(x[1]*x[2]) - 4x[1] - 6x[2]
        ∇f(x) = ∇(f,x)
        ∇²f(x) = ∇²(f,x)

        g₁(x) = x[1]^2 - x[2]   # ≦ 0
        g₂(x) = x[1] + 5x[2] - 5 # ≦ 0
        g₃(x) = -x[1] # ≦ 0
        g₄(x) = -x[2] # ≦ 0

        ∇g₁(x) = ∇(g₁,x)
        ∇²g₁(x) = ∇²(g₁,x)
        ∇g₂(x) = ∇(g₂,x)
        ∇g₃(x) = ∇(g₃,x)
        ∇g₄(x) = ∇(g₄,x)

        L(x,u) = f(x) + u[1]*g₁(x) + u[2]*g₂(x) + u[3]*g₃(x) + u[4]*g₄(x)
        ∇²L(x,u) = ∇²f(x) + u[1]*∇²g₁(x)  # g₂, g₃, and g₄ vanish as they are linear .

# Initialisation

        k = 1             # iter count
        N = 10            # max iterations
        ϵ = 1e-6          # tolerance
        xᵏ = [0.5; 0.5]   # starting point
        uᵏ = zeros(4)     # initial dual values
        x = zeros(2, N)   # trajectory
        x[:,k] = xᵏ

        ## Main loop
        for k = 1:N-1
                xᵏ = x[:,k]
                # Calculate ∇f, ∇h, and ∇L²
                ∇fᵏ = ∇f(xᵏ)
                ∇gᵏ = [∇g₁(xᵏ), ∇g₂(xᵏ), ∇g₃(xᵏ), ∇g₄(xᵏ)]
                ∇L²ᵏ = ∇²L(xᵏ,uᵏ)
                gᵏ = [g₁(xᵏ), g₂(xᵏ), g₃(xᵏ), g₄(xᵏ)]

                # Direction search subproblem
                QP = Model()
                set_optimizer(QP, Ipopt.Optimizer)
                set_silent(QP)
                @variable(QP, d[1:n])
                @objective(QP, Min, dot(∇fᵏ,d) + 0.5*(d'*∇L²ᵏ*d))
                @constraint(QP, LinearIneq[i=1:m], dot(∇gᵏ[i],d) + gᵏ[i] <= 0)

                optimize!(QP)

                dᵏ = value.(d)           # obtain direction
                x[:,k+1] = xᵏ + dᵏ       # step
                uᵏ = dual.(LinearIneq)   # obtain dual values

                if norm(x[:,k+1] - x[:,k]) < ϵ # stop condition
                        x = x[:,1:k+1]
                        return x
                end
        end
end

SQP (generic function with 1 method)

In [3]:
f(x) =  2x[1]^2 + 2x[2]^2 - 2(x[1]*x[2]) - 4x[1] - 6x[2]
x = SQP()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



2×6 Matrix{Float64}:
 0.5  1.04167   0.913169  0.905021  0.904988  0.904988
 0.5  0.791667  0.817366  0.818996  0.819002  0.819002

In [8]:
## Plotting...
n = 1000
x1 = range(0,5,length=n)
x2 = range(0,5,length=n)
z = [f([x1[i],x2[j]]) for j=1:n, i=1:n]

contour(x1, x2, z,
        clims = (-10,-5))
        #clabels = true)
plot!(x1, x1.^2, fill=(10,0.1), color=3, label=L"$x_1 + 5x_2 \leq 5$")
plot!(x1, 1 .- (1/5)*x1, fill=(0,0.1), color=2,
    xaxis = (L"$x_1$", (0,2)),
    yaxis = (L"$x_2$", (0,2)),
    aspect_ratio = :equal,
    colorbar = false,
    label= L"$x_1^2 - x2 \leq 0$",
    legend = true)

plot!(x[1,:], x[2,:], marker = :o, color=1, label="Trajectory")

# First order approximation at initial point for constraint 1
plot!(x1, -0.25 .+ x1, color=3, line = :dash, label=L"$g(x_0) + \nabla g(x_0)^\top x$") # dot(∇gᵏ[i],d) + gᵏ[i] = 0

annotate!(0.5, 0.4, text(L"$x_0$",9,:bottom))

savefig("SQP_example.pdf")

sys:1: UserWarning: The following kwargs were not used by contour: 'label'


In [5]:
# n = 10
# x1 = range(0,5,length=n)
# x2 = range(0,5,length=n)
# surface(x1, x2, (x,y)-> x^2-y)